In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import
import pandas as pd
from pprint import pprint
from zen_garden.postprocess.results import Results
from pathlib import Path
from os import chdir

# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [12]:
# define import path
path_import = Path("../data/outputs/05_CCTS_WTE_small_01_cost_optimal")

# define export path
path_export = Path("/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/05_CCTS_WTE_small_02_n1")

In [13]:
# cost optimal solution
results = Results(path_import)

In [14]:
# get flow conversion and save as csv
results.get_df('flow_conversion_input', to_csv="flow_conversion_input")

technology         carrier           node              time_operation
capture_amine      flue_gas          swiss_storage_CH  0                  0.000000
                                     waste_11_CH       0                 24.844685
                                     waste_19_CH       0                 33.982036
                                     waste_1_CH        0                 29.319244
                                     waste_2_CH        0                 21.518910
                                                                           ...    
permanent_storage  co2_liquid_16bar  waste_11_CH       0                  0.000000
                                     waste_19_CH       0                  0.000000
                                     waste_1_CH        0                  0.000000
                                     waste_2_CH        0                  0.000000
                                     waste_9_CH        0                  0.000000
Name: flow_conver

In [15]:
# get flow conversion csv
df = pd.read_csv('flow_conversion_input.csv')

In [16]:
# create nominal flow conversion files

# find unique technologies
technology = df['technology'].unique()

for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path(path_export)
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/nominal_flow_conversion_input.csv')
    
    #Clean up table
    x = x.drop(columns='technology')
    x = x[x['flow_conversion_input'] != 0]
    x = x.rename(columns={'time_operation': 'year'})
    x['year'] = x['year'] + 2025
    
    # Pivot the DataFrame
    x = x.pivot(index=['node', 'year'], columns='carrier', values='flow_conversion_input').reset_index()
    x.columns.name = None  # Remove the 'carrier' label on columns
    
    #clean up table, differentiate between node and edge
    if path.parent.name == 'set_transport_technologies':
        x = x.rename(columns={'location': 'edge'})

    else:
        x = x.rename(columns={'location': 'node'})

    #Safe
    x.to_csv(filepath, index=False)